<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Important point - please read the functionality</h2>
            <span style="color:#ff7800;">This code asks for User request and then asks 4 LLMS (openAi, Gemini, Groq and Ollama) for a response for the user request </br> Then it evalutes each of the response with the 4 LLMS and grades them and finally displays the grading as a HTML Table
            </span>
        </td>
    </tr>
</table>

In [ ]:
# Start with imports - ask ChatGPT to explain any package that you don't know

import os
import sys
import pandas as pd
import json
import asyncio
import ipywidgets as widgets
from dotenv import load_dotenv
from openai import OpenAI
from anthropic import Anthropic
from IPython.display import Markdown, HTML, display, clear_output

In [ ]:
# Always remember to do this!
load_dotenv(override=True)

In [ ]:
# function to get multiline input from the user as the request

def multiline_input_widget_auto(
    var_name="user_text",
    title="Enter your challenging request here:",
    placeholder="Type your request here…",
    button_text="Submit",
    height="200px",
    width="100%"
):
    ta = widgets.Textarea(
        value="",
        placeholder=placeholder,
        layout=widgets.Layout(width=width, height=height)
    )
    btn = widgets.Button(description=button_text)
    out = widgets.Output()

    def on_click(_):
        globals()[var_name] = ta.value  # Save to a Python variable
        with out:
            clear_output()
            print(f"✅ Saved to variable '{var_name}'")

    btn.on_click(on_click)
    display(widgets.VBox([widgets.HTML(f"<b>{title}</b>"), ta, btn, out]))

In [ ]:
# Get the user request"
multiline_input_widget_auto("user_request") 

In [ ]:
#Replay the user_request
print ("Your request entered is :")
print (user_request)

In [ ]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

In [ ]:
# Get input from the user as to the challenging question he would like to have an best answer for


messages = [{"role": "user", "content": user_request}]

In [ ]:
#Define Class to store models and its answers
class ModelResponseStorage:
    def __init__(self):
        # Internal storage for all pairs
        self.data = []

    def add(self, ai_model, response):
        """Add a pair of text values to storage."""
        self.data.append([ai_model, response])
        #print(f"✅ Added: [{text1}, {text2}]")

    def get_all(self):
        """Return all stored pairs as a list."""
        return self.data

    def get_first_values(self):
        """Return a list of all first values in the pairs."""
        return [pair[0] for pair in self.data]

    def get_second_values(self):
        """Return a list of all second values in the pairs."""
        return [pair[1] for pair in self.data]

    def display(self):
        """Pretty print all stored pairs with an index."""
        for idx, (first, second) in enumerate(self.data, start=1):
            print(f"{idx}: {first} — {second}")

    def unzip(self):
        """Return two separate lists: first values, second values."""
        if not self.data:
            return [], []
        first_values, second_values = zip(*self.data)
        return list(first_values), list(second_values)

In [ ]:
#instantiate the class to store the values
store_response= ModelResponseStorage ()

In [ ]:
#ask the User request to the OpenAI and store the response for evaluation
ai_model="gpt-4o-mini"
openai = OpenAI()
response = openai.chat.completions.create(
    model=ai_model,
    messages=messages,
)
answer = response.choices[0].message.content
#Store the model and its answer in our array
store_response.add(ai_model,answer)

In [ ]:
#ask the User request to the Gemini and store the response for evaluation
ai_model = "gemini-2.0-flash"
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
response = gemini.chat.completions.create(model=ai_model, messages=messages)
answer = response.choices[0].message.content

#Store the model and its answer in our array
store_response.add(ai_model,answer)

In [ ]:
#ask the User request to the Groq and store the response for evaluation

ai_model = "llama-3.3-70b-versatile"
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")

response = groq.chat.completions.create(model=ai_model, messages=messages)
answer = response.choices[0].message.content

#Store the model and its answer in our array
store_response.add(ai_model,answer)


In [ ]:
#ask the User request to the ollama and store the response for evaluation

ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
ai_model = "llama3.2"

response = ollama.chat.completions.create(model=ai_model, messages=messages)
answer = response.choices[0].message.content

#Store the model and its answer in our array
store_response.add(ai_model,answer)

In [ ]:
# Retrive the information from the Class Instance

competing_models = store_response.get_first_values()
model_answers = store_response.get_second_values()

In [ ]:
# Let's bring this together - note the use of "enumerate"

together = ""
for index, answer in enumerate(model_answers):
    together += f"# Response from competitor {index+1}\n\n"
    together += answer + "\n\n"

In [ ]:
#print(together)

In [ ]:
judge = f"""You are judging a best response given by {len(competing_models)} competitors.
Each model has been given this question:

{user_request}

Your job is to evaluate each response for clarity and strength of argument, and rank them in order of best to worst.
Respond with JSON, and only JSON, with the following format:
{{"results": ["Rank1", "Rank2", "Rank3", ...]}}

Here are the responses from each competitor:

{together}

Now respond with the JSON with the ranked order of the competitors, nothing else. Do not include markdown formatting or code blocks."""


In [ ]:
#print(judge)

In [ ]:
judge_messages = [{"role": "user", "content": judge}]

In [ ]:
# Judgement time!
#using OpenAI model gpt-4o-mini
eval_model="gpt-4o-mini"
openai = OpenAI()
response = openai.chat.completions.create(
    model= eval_model,
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)


In [ ]:
# OK let's turn this into results!

results_dict = json.loads(results)
ranks = results_dict["results"]
#print(ranks)
row1string = eval_model 
#print("Juding Using Model:"+ eval_model)
for index, result in enumerate(ranks):
    number = int(result[-1])-1
    competitor = competing_models[number]
    answer = model_answers[number]
    row1string += "," + competitor

In [ ]:
# Judgement time!
#using Gemini model gemini-2.0-flash
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")
eval_model = "gemini-2.0-flash"
response = gemini.chat.completions.create(
    model=eval_model,
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)

In [ ]:
results_dict = json.loads(results)
ranks = results_dict["results"]
#print(ranks)
row2string = eval_model 
#print("Juding Using Model:"+ eval_model)
for index, result in enumerate(ranks):
    number = int(result[-1])-1
    #print(number)
    competitor = competing_models[number]
    #print(competitor)
    answer = model_answers[number]
    row2string += "," + competitor

In [ ]:
# Judgement time!
#using Groq
groq = OpenAI(api_key=groq_api_key, base_url="https://api.groq.com/openai/v1")
eval_model = "llama-3.3-70b-versatile"
response = groq.chat.completions.create(
    model=eval_model,
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)

In [ ]:
results_dict = json.loads(results)
ranks = results_dict["results"]
#print(ranks)
row3string = eval_model 
#print("Juding Using Model:"+ eval_model)
for index, result in enumerate(ranks):
    number = int(result[-1])-1
    #print(number)
    competitor = competing_models[number]
    #print(competitor)
    answer = model_answers[number]
    row3string += "," + competitor

In [ ]:
# Judgement time!
#using ollama
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
eval_model = "llama3.2"
response = ollama.chat.completions.create(
    model=eval_model,
    messages=judge_messages,
)
results = response.choices[0].message.content
print(results)

In [ ]:
results_dict = json.loads(results)
ranks = results_dict["results"]
#print(ranks)
row4string = eval_model 
#print("Juding Using Model:"+ eval_model)
for index, result in enumerate(ranks):
    number = int(result[-1])-1
    competitor = competing_models[number]
    answer = model_answers[number]
    row4string += "," + competitor

In [ ]:
HeaderRow="Eval_Model" + "," +  "Rank1"+ "," + "Rank2"+ "," + "Rank3"+ "," + "Rank4"
#print(HeaderRow)
#print(row1string)
#print(row2string)
#print(row3string)
#print(row4string)

#Build a table using Pandas Table
rows = [HeaderRow, row1string, row2string, row3string, row4string]

# Split each row into a list of values
data = [row.split(",") for row in rows]

# Create a DataFrame: first row = column names, rest = data
df = pd.DataFrame(data[1:], columns=data[0])

#display table as html in this book
display(HTML(df.to_html(index=False)))